## Gradient Descent Code-Along

Let's walk through how gradient descent works using code.

In [2]:
import pandas as pd
import numpy as np

In [3]:
np.random.seed(42)

In [4]:
temp = np.random.uniform(-10, 80, 100)

In [5]:
temp[0:10]

array([23.7086107 , 75.56428758, 55.87945476, 43.87926358,  4.04167764,
        4.03950683, -4.7724749 , 67.95585312, 44.10035106, 53.726532  ])

In [6]:
print(np.mean(temp), np.std(temp, ddof=1))

32.31626690403884 26.77404699137873


**Ohio State Fun Facts:**
1. Ohio Stadium can seat 104,944 people. (Source: [Wikipedia](https://en.wikipedia.org/wiki/Ohio_Stadium).)
2. Ohio Stadium's record attendance is 110,045 people. (Source: [Wikipedia](https://en.wikipedia.org/wiki/Ohio_Stadium).)
3. Michigan sucks. (Source: It's just a fact.)
4. Ohio State students enjoy alcohol. (Source: first-hand knowledge.)

In [7]:
beers_sold = 200000 + 1000 * temp + np.random.normal(0, 20000)

In [8]:
beers_sold[:10]

array([225449.55206103, 277305.22894166, 257620.39612779, 245620.2049425 ,
       205782.61900458, 205780.44819502, 196968.4664599 , 269696.79448451,
       245841.29242165, 255467.47336641])

$$ \text{beers_sold}_i = 200000 + 1000 * \text{temp}_i + \varepsilon_i $$

In [9]:
df = pd.DataFrame.from_dict({'temp': temp,
                             'beers_sold': beers_sold})

In [10]:
df.head()

,beers_sold,temp
0,225449.552061,23.708611
1,277305.228942,75.564288
2,257620.396128,55.879455
3,245620.204942,43.879264
4,205782.619005,4.041678


#### Our goal is to fit a model here.
- You and I know that our $y$-intercept $\beta_0$ is 200,000.
- You and I know that our slope $\beta_1$ is 1,000.
- However, our computer does not know that. Our computer has to estimate $\hat{\beta}_0$ and $\hat{\beta}_1$ from the data.
    - We might say that our **machine** has to... **learn**.

#### Our workflow:
1. Instantiate model.
2. Select a learning rate $\alpha$.
3. Select a starting point $\hat{\beta}_{1,0}$.
4. Calculate the gradient of the loss function.
5. Calculate $\hat{\beta}_{1,i+1} = \hat{\beta}_{1,i} - \alpha * \frac{\partial L}{\partial \beta_1}$.
6. Check value of $|\hat{\beta}_{1,i+1} - \hat{\beta}_{1,i}|$.
7. Repeat steps 4 through 6 until "stopping condition" is met.

#### Step 1. Instantiate model.

Our model takes on the form:
$$ Y = \beta_0 + \beta_1 X + \varepsilon$$

#### Step 2. Select a learning rate $\alpha$.

$$\alpha = 0.1$$

In [11]:
alpha = 0.1

#### Step 3. Select a starting point.
The zero-th iteration of $\hat{\beta}_1$ is going to start at, say, 20.
$$\hat{\beta}_{1,0} = 20$$

Two points:
- You and I know that the true value of $\beta_1$ is 1000. We need the computer to figure that part out!
- We're going to pretend like the computer already knows the value for $\beta_0$. In reality, we'd have to do this for $\beta_0$ and for $\beta_1$ at the same time.

In [12]:
beta_1 = 20

#### Step 4. Calculate the gradient of the loss function with respect to parameter $\beta_1$.

The loss function, $L$, is our mean square error.

$$L = \sum_{i = 1} ^ n (y_i - \hat{y}_i)^2 $$

$$\Rightarrow L = \sum_{i = 1} ^ n (y_i - (\hat{\beta}_0 + \hat{\beta}_1x_i))^2 $$

The gradient of this loss function with respect to $\beta_1$ is:

$$\frac{\partial L}{\partial \beta_1} = \frac{2}{n} \sum_{i=1}^n -x_i(y_i - (\hat{\beta}_1x_i + \hat{\beta}_0)) $$

In [14]:
def beta_1_gradient(x, y, beta_1, beta_0):
    n = len(x)
    gradient = 0
    
    for i in range(n):
        gradient += -1 * x[i] * (y[i] - (beta_1 * x[i] + beta_0))
    gradient *= (2 / n)
    return(gradient)

#### Step 5. Calculate $\hat{\beta}_{1,i+1} = \hat{\beta}_{1,i} - \alpha * \frac{\partial L}{\partial \beta_1}$.

In [15]:
def update_beta_1(beta_1, alpha, gradient):
    beta_1 = beta_1 - alpha * gradient
    return(beta_1)

#### Step 6. Check value of $|\hat{\beta}_{1,i+1} - \hat{\beta}_{1,i}|$.

In [16]:
def check_update(beta_1, updated_beta_1, tolerance = 0.1):
    if abs(beta_1 - updated_beta_1) < tolerance:
        return(True)
    else:
        return(False)

#### Step 7: Save final value of $\hat{\beta}_1$.

#### Putting it all together...

In [18]:
def gradient_descent(x, y, beta_1 = 0, alpha = 0.01, max_iter = 100):
    converged = False
    for i in range(max_iter):
        gradient = beta_1_gradient(x, y, beta_1, 200000)
        updated_beta_1 = update_beta_1(beta_1, alpha, gradient)
        converged = check_update(beta_1, updated_beta_1)
        beta_1 = updated_beta_1
        if converged == True:
            print("Our algorithm converged after " + str(i) + " iterations with a beta_1 value of: " + str(beta_1))
            break
        print("Iteration " + str(i) + " with beta_1 value of: " + str(beta_1))
    if converged == False:
        print("Our algorithm did not converge, so do not trust the value of beta_1.")
    return(beta_1)

In [23]:
gradient_descent(df['temp'], # X data
                 df['beers_sold'], # Y data
                 beta_1 = 20, # initial guess for beta_1
                 alpha = 0.00002, # learning rate
                 max_iter = 100) # maximum iterations

Iteration 0 with beta_1 value of: 91.0080993893964
Iteration 1 with beta_1 value of: 157.0342074619574
Iteration 2 with beta_1 value of: 218.42786515307563
Iteration 3 with beta_1 value of: 275.5140892956825
Iteration 4 with beta_1 value of: 328.5950932530571
Iteration 5 with beta_1 value of: 377.95188683050935
Iteration 6 with beta_1 value of: 423.84576393583836
Iteration 7 with beta_1 value of: 466.51968586421185
Iteration 8 with beta_1 value of: 506.19956753054987
Iteration 9 with beta_1 value of: 543.0954734587023
Iteration 10 with beta_1 value of: 577.4027298589632
Iteration 11 with beta_1 value of: 609.3029586812396
Iteration 12 with beta_1 value of: 638.9650391181309
Iteration 13 with beta_1 value of: 666.5460016480988
Iteration 14 with beta_1 value of: 692.1918593517744
Iteration 15 with beta_1 value of: 716.0383809023747
Iteration 16 with beta_1 value of: 738.2118093224249
Iteration 17 with beta_1 value of: 758.8295303118708
Iteration 18 with beta_1 value of: 778.0006936856979

1030.8200111561735